In [ ]:
# Install if necessary
print('Installing packages')
!pip install datasets==1.18.3 
!pip install transformers[sentencepiece]==4.16.2 
!pip install tweet-preprocessor
!pip install pyspellchecker
!pip install optuna

Installing packages
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 311 kB 8.4 MB/s 
     |████████████████████████████████| 212 kB 57.2 MB/s 
     |████████████████████████████████| 182 kB 59.5 MB/s 
     |████████████████████████████████| 132 kB 13.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 7.3 MB/s 
     |████████████████████████████████| 880 kB 68.2 MB/s 
     |████████████████████████████████| 7.6 MB 49.6 MB/s 
     |████████████████████████████████| 1.3 MB 56.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3ee5301706de13177f298fda9a001623e56df5394b3038be5254267ae9648182
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: htt

In [ ]:
!gdown 1U6CU4VonapD40kAkK3_X6LirG1_f8nim
!gdown 1gBqHkxh5Bg4O_VIEzrLO-mG9tkZ8uD_F

Downloading...
From: https://drive.google.com/uc?id=1U6CU4VonapD40kAkK3_X6LirG1_f8nim
To: /content/H1_Offensive_Language_Identification_test.csv
100% 132k/132k [00:00<00:00, 86.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gBqHkxh5Bg4O_VIEzrLO-mG9tkZ8uD_F
To: /content/H1_Offensive_Language_Identification_train.csv
100% 1.83M/1.83M [00:00<00:00, 198MB/s]


In [ ]:
model_checkpoint = "GroNLP/hateBERT"
# model_checkpoint = "mrm8488/distilroberta-finetuned-tweets-hate-speech"
# model_checkpoint = "cardiffnlp/twitter-roberta-base-offensive"
# model_checkpoint = "cardiffnlp/twitter-roberta-base-hate"
# model_checkpoint = "Hate-speech-CNERG/english-abusive-MuRIL"
batch_size = 8

<h2>Text Preprocessing and Dataset Preperation

In [ ]:
import pandas as pd
tweets_df_train = pd.read_csv("/content/H1_Offensive_Language_Identification_train.csv")
tweets_df_test = pd.read_csv("/content/H1_Offensive_Language_Identification_test.csv")

In [ ]:
import preprocessor as p
import numpy as np

# Remove numbers, emojis and &'s
p.set_options(p.OPT.NUMBER, p.OPT.EMOJI)

tweets_df_train_cleaned = (tweets_df_train
                           .assign(tweet=tweets_df_train["tweet"].apply(p.clean).str.replace("&", "and ").str[:512],
                                   label=np.where(tweets_df_train.label == "OFF", 0, 1) # Change OFF to 1 and NOT to 0
                 )
             )


tweets_df_test_cleaned = (tweets_df_test
             .assign(
                 tweet=tweets_df_test["tweet"].apply(p.clean).str.replace("&", "and ").str[:512],
                 )
             )

In [ ]:
import re
import nltk
import string
import pandas as pd
from textblob import Word
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove("not")
stop_words.remove("no")
pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')

html = re.compile('<.*?>')
spell = SpellChecker()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def preprocess(text, stop_word_pattern = pattern, html_pattern = html):
    text = re.sub("([@][A-Za-z0-9_]+)|(\w+:\/\/\S+)", "", text)
    text = re.sub("\s+"," ",text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    text = re.sub(html_pattern, '', text)
    text = re.sub(r"URL", '', text)
    text = re.sub("\s+"," ",text)
    return text.lower()

In [ ]:
cleaned = [preprocess(sent) for sent in tweets_df_train_cleaned["tweet"]]
tweets_df_train_cleaned["tweet"] = cleaned

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(tweets_df_train_cleaned, test_size=0.065, random_state=42)
print(df_train.shape, df_test.shape)

train_data = Dataset.from_pandas(df_train)
test_data = Dataset.from_pandas(df_test)
print(train_data)

(12379, 3) (861, 3)
Dataset({
    features: ['tweet', 'label', 'id', '__index_level_0__'],
    num_rows: 12379
})


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["tweet"], truncation=True)

In [ ]:
train_dataset = train_data.map(preprocess_function, batched=True)
test_dataset = test_data.map(preprocess_function, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_metric
# Load metric
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

metric_name = "f1"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

<h2>Hyper Parameter Search

In [ ]:
def model_init():
  # Load model from checkpoint
  model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

  # Set the device automatically (GPU or CPU)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # Move model to device
  model.to(device)
  
  return model

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args = args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize")

In [ ]:
best_run

BestRun(run_id='3', objective=1.6409292337964805, hyperparameters={'learning_rate': 4.90357213166462e-05, 'num_train_epochs': 1, 'seed': 39, 'per_device_train_batch_size': 32})

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

In [ ]:
trainer.push_to_hub(f"{model_name}-finetuned")